In [3]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

In [4]:
slice = fablib.new_slice(name='dedicated_path3')

In [5]:
site1 = 'WASH'
site2 = 'MAX'
site3 = 'TACC'
site4 = 'UCSD'

print (f'The selected sites are {site1}, {site2}, {site3} {site4}') 

The selected sites are WASH, MAX, TACC UCSD


In [6]:
client = slice.add_node(name="client", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

server = slice.add_node(name="server", 
                      site=site3, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

In [7]:
client_iface = client.add_component(model='NIC_Basic', name = 'clientToSwitch1').get_interfaces()[0]
server_iface = server.add_component(model='NIC_Basic', name = 'serverToSwitch1').get_interfaces()[0]

client_iface2 = client.add_component(model='NIC_Basic', name = 'clientToSwitch2').get_interfaces()[0]
# server_iface2 = server.add_component(model='NIC_Basic', name = 'serverToSwitch2').get_interfaces()[0]

client_iface3 = client.add_component(model='NIC_Basic', name = 'clientToSwitch3').get_interfaces()[0]
server_iface3 = server.add_component(model='NIC_Basic', name = 'serverToSwitch3').get_interfaces()[0]

In [8]:
# Add a node
switch1 = slice.add_node(name="switch1", 
                      site=site2, 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

switch2 = slice.add_node(name="switch2", 
                      site=site4, 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

switch3 = slice.add_node(name="switch3", 
                      site=site4, 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

In [9]:
switch1_iface1 = switch1.add_component(model='NIC_Basic', name='net1_nic').get_interfaces()[0]
switch1_iface2 = switch1.add_component(model='NIC_Basic', name='net2_nic').get_interfaces()[0]
switch1_iface3 = switch1.add_component(model='NIC_Basic', name='net3_nic').get_interfaces()[0]

switch2_iface1 = switch2.add_component(model='NIC_Basic', name='net1_nic').get_interfaces()[0]
switch2_iface2 = switch2.add_component(model='NIC_Basic', name='net2_nic').get_interfaces()[0]

switch3_iface1 = switch3.add_component(model='NIC_Basic', name='net1_nic').get_interfaces()[0]
switch3_iface2 = switch3.add_component(model='NIC_Basic', name='net2_nic').get_interfaces()[0]

In [10]:
net1 = slice.add_l2network(name='net1', interfaces=[client_iface, switch1_iface1])

In [11]:
net2 = slice.add_l2network(name='net2', interfaces=[switch1_iface2, server_iface])

In [12]:
# net3 = slice.add_l2network(name='net3', interfaces=[server_iface2, switch2_iface1])

In [13]:
net3 = slice.add_l2network(name='net3', interfaces=[switch2_iface2, switch3_iface1])

In [14]:
net4 = slice.add_l2network(name='net4', interfaces=[switch3_iface2, client_iface2])

In [15]:
net5 = slice.add_l2network(name='net5', interfaces=[switch1_iface3, switch2_iface1])

In [16]:
#Submit Slice Request
slice.submit();


Retry: 11, Time: 534 sec


ID,ee060b86-357f-4a59-adde-98f5986570f7
Name,dedicated_path3
Lease Expiration (UTC),2023-12-06 05:01:36 +0000
Lease Start (UTC),2023-12-05 05:01:36 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
54623638-f421-45be-ac38-6c604da54458,client,4,8,100,default_ubuntu_20,qcow2,wash-w2.fabric-testbed.net,WASH,ubuntu,2001:400:a100:3020:f816:3eff:fe97:ec40,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fe97:ec40,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
87838338-af8b-4ea6-9dcb-a45b41ef7129,server,4,8,100,default_ubuntu_20,qcow2,tacc-w5.fabric-testbed.net,TACC,ubuntu,129.114.110.84,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.84,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
837f1104-fa8e-4b52-a948-6105b4835a27,switch1,32,16,100,default_ubuntu_20,qcow2,max-w5.fabric-testbed.net,MAX,ubuntu,63.239.135.97,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@63.239.135.97,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
ea71f56b-ba8e-476a-ba59-a0559ebbaaf1,switch2,32,16,100,default_ubuntu_20,qcow2,ucsd-w3.fabric-testbed.net,UCSD,ubuntu,132.249.252.169,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.169,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
72b1c965-2ec7-4d44-ae87-a2cca434cfc5,switch3,32,16,100,default_ubuntu_20,qcow2,ucsd-w3.fabric-testbed.net,UCSD,ubuntu,132.249.252.190,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.190,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
ef2bd825-2ddd-47e3-aa8b-dafac2dabcfa,net1,L2,L2STS,None,None,None,Active,
e7012bda-92ff-44c6-aae0-d42c27fbd485,net2,L2,L2STS,None,None,None,Active,
dfaf8ce2-ff68-4356-b5a4-b7e353240ded,net3,L2,L2Bridge,UCSD,None,None,Active,
60609943-c44c-452f-84c9-f146bfbecc9f,net4,L2,L2STS,None,None,None,Active,
40c4882e-0ba7-48d5-89ca-97864114c238,net5,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
switch1-net1_nic-p1,p1,switch1,net1,100,config,,02:54:AB:58:99:C9,enp8s0,enp8s0,None,4
switch1-net2_nic-p1,p1,switch1,net2,100,config,,02:AC:DB:D1:32:A1,enp9s0,enp9s0,None,4
switch1-net3_nic-p1,p1,switch1,net5,100,config,,02:27:59:CB:14:44,enp7s0,enp7s0,None,4
switch2-net1_nic-p1,p1,switch2,net5,100,config,,06:CE:FC:94:3E:55,enp7s0,enp7s0,None,4
switch2-net2_nic-p1,p1,switch2,net3,100,config,,06:AE:3B:35:9F:83,enp6s0,enp6s0,None,4
switch3-net2_nic-p1,p1,switch3,net4,100,config,,0A:E9:2C:69:4F:9A,enp7s0,enp7s0,None,4
switch3-net1_nic-p1,p1,switch3,net3,100,config,,0A:20:B9:BE:1A:AC,enp6s0,enp6s0,None,4
client-clientToSwitch3-p1,p1,client,None,100,config,,12:38:6C:3E:48:39,enp7s0,enp7s0,None,4
client-clientToSwitch2-p1,p1,client,net4,100,config,,12:75:55:C3:E5:F2,enp8s0,enp8s0,None,4
client-clientToSwitch1-p1,p1,client,net1,100,config,,12:AE:B8:3A:19:53,enp9s0,enp9s0,None,4



Time to print interfaces 553 seconds


In [17]:
# slice = fablib.get_slice(name = 'dedicated_path3')

switch1 = slice.get_node(name="switch1")     
switch1.upload_file('scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch1.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

switch2 = slice.get_node(name="switch2")     
switch2.upload_file('scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch2.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

switch3 = slice.get_node(name="switch3")     
switch3.upload_file('scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch3.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

In [18]:
client = slice.get_node(name="client")
server = slice.get_node(name="server")
stdout, stderr = client.execute(f'sudo apt-get install -y ffmpeg', quiet=True)
stdout, stderr = client.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = server.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = switch1.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = switch2.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = switch3.execute(f'sudo apt-get install -y net-tools', quiet=True)

In [19]:
stdout, stderr = client.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)
stdout, stderr = server.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)
stdout, stderr = switch1.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)
stdout, stderr = switch2.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)
stdout, stderr = switch3.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)

In [20]:
node1_iface = client.get_interface(network_name='net1') 
client_iface_name = node1_iface.get_device_name()
print(f'client_iface: {client_iface_name}')

node2_iface = server.get_interface(network_name='net2') 
server_iface_name = node2_iface.get_device_name()
print(f'server_iface: {server_iface_name}')

switch1_iface1 = switch1.get_interface(network_name='net1') 
switch1_iface1_name = switch1_iface1.get_device_name()
print(f'switch1_iface1: {switch1_iface1_name}')

switch1_iface2 = switch1.get_interface(network_name='net2') 
switch1_iface2_name = switch1_iface2.get_device_name()
print(f'switch1_iface2: {switch1_iface2_name}')

# switch2_iface1 = switch2.get_interface(network_name='net3') 
# switch2_iface1_name = switch2_iface1.get_device_name()
# print(f'switch2_iface1: {switch2_iface1_name}')

# server_iface2 = server.get_interface(network_name='net3') 
# server_iface2_name = server_iface2.get_device_name()
# print(f'server_iface2: {server_iface2_name}')

switch2_iface2 = switch2.get_interface(network_name='net3') 
switch2_iface2_name = switch2_iface2.get_device_name()
print(f'switch2_iface2: {switch2_iface2_name}')

switch3_iface1 = switch3.get_interface(network_name='net3') 
switch3_iface1_name = switch3_iface1.get_device_name()
print(f'switch3_iface1: {switch3_iface1_name}')

switch3_iface2 = switch3.get_interface(network_name='net4') 
switch3_iface2_name = switch3_iface2.get_device_name()
print(f'switch3_iface2: {switch3_iface2_name}')

client_iface2 = client.get_interface(network_name='net4') 
client_iface2_name = client_iface2.get_device_name()
print(f'client_iface2: {client_iface2_name}')

switch2_iface1 = switch2.get_interface(network_name='net5') 
switch2_iface1_name = switch2_iface1.get_device_name()
print(f'switch2_iface1: {switch2_iface1_name}')

switch1_iface3 = switch1.get_interface(network_name='net5') 
switch1_iface3_name = switch1_iface3.get_device_name()
print(f'switch1_iface3: {switch1_iface3_name}')

client_iface: enp9s0
server_iface: enp8s0
switch1_iface1: enp8s0
switch1_iface2: enp9s0
switch2_iface2: enp6s0
switch3_iface1: enp6s0
switch3_iface2: enp7s0
client_iface2: enp8s0
switch2_iface1: enp7s0
switch1_iface3: enp7s0


In [21]:
stdout, stderr = client.execute(f'sudo ip link set dev {client_iface_name} up', quiet=True)
stdout, stderr = server.execute(f'sudo ip link set dev {server_iface_name} up', quiet=True)
# stdout, stderr = server.execute(f'sudo ip link set dev {server_iface2_name} up', quiet=True)
stdout, stderr = client.execute(f'sudo ip link set dev {client_iface2_name} up', quiet=True)
stdout, stderr = switch1.execute(f'sudo ip link set dev {switch1_iface1_name} up', quiet=True)
stdout, stderr = switch1.execute(f'sudo ip link set dev {switch1_iface2_name} up', quiet=True)
stdout, stderr = switch1.execute(f'sudo ip link set dev {switch1_iface3_name} up', quiet=True)
stdout, stderr = switch2.execute(f'sudo ip link set dev {switch2_iface1_name} up', quiet=True)
stdout, stderr = switch2.execute(f'sudo ip link set dev {switch2_iface2_name} up', quiet=True)
stdout, stderr = switch3.execute(f'sudo ip link set dev {switch3_iface1_name} up', quiet=True)
stdout, stderr = switch3.execute(f'sudo ip link set dev {switch3_iface2_name} up', quiet=True)

In [22]:
client_iface_MAC = '00:00:00:00:00:01'
server_iface_MAC = '00:00:00:00:00:02'

client_iface2_MAC = '00:00:00:00:00:03'
# server_iface2_MAC = '00:00:00:00:00:04'

switch1_iface1_MAC = '00:00:00:00:00:05'
switch1_iface2_MAC = '00:00:00:00:00:06'
switch1_iface3_MAC = '00:00:00:00:00:04'

switch2_iface1_MAC = '00:00:00:00:00:07'
switch2_iface2_MAC = '00:00:00:00:00:08'

switch3_iface1_MAC = '00:00:00:00:00:09'
switch3_iface2_MAC = '00:00:00:00:00:10'

In [23]:
client = slice.get_node(name="client")     

client_switch1_subnet = "192.168.1.0/24"
client_ip1 = '192.168.1.10/24'
switch1_ip1 = '192.168.1.1/24'

stdout, stderr = client.execute(f'sudo ifconfig {client_iface_name} {client_ip1}')
stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface1_name} {switch1_ip1}')

stdout, stderr = client.execute(f'sudo ifconfig {client_iface_name} hw ether {client_iface_MAC}')
stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface1_name} hw ether {switch1_iface1_MAC}')

In [24]:
server = slice.get_node(name="server")

server_switch1_subnet = "192.168.2.0/24"
server_ip1 = '192.168.2.10/24'
switch1_ip2 = '192.168.2.1/24'

stdout, stderr = server.execute(f'sudo ifconfig {server_iface_name} {server_ip1}')
stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface2_name} {switch1_ip2}')

stdout, stderr = server.execute(f'sudo ifconfig {server_iface_name} hw ether {server_iface_MAC}')
stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface2_name} hw ether {switch1_iface2_MAC}')

In [25]:
client = slice.get_node(name="client")

client_switch3_subnet = "192.168.3.0/24"
client_ip2 = '192.168.3.10/24'
switch3_ip2 = '192.168.3.1/24'

stdout, stderr = client.execute(f'sudo ifconfig {client_iface2_name} {client_ip2}')
stdout, stderr = switch3.execute(f'sudo ifconfig {switch3_iface2_name} {switch3_ip2}')

stdout, stderr = client.execute(f'sudo ifconfig {client_iface2_name} hw ether {client_iface2_MAC}')
stdout, stderr = switch3.execute(f'sudo ifconfig {switch3_iface2_name} hw ether {switch3_iface2_MAC}')

In [26]:
switch1 = slice.get_node(name="switch1")

switch1_switch2_subnet = "192.168.4.0/24"
switch1_ip3 = '192.168.4.10/24'
switch2_ip1 = '192.168.4.1/24'

stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface3_name} {switch1_ip3}')
stdout, stderr = switch2.execute(f'sudo ifconfig {switch2_iface1_name} {switch2_ip1}')

stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface3_name} hw ether {switch1_iface3_MAC}')
stdout, stderr = switch2.execute(f'sudo ifconfig {switch2_iface1_name} hw ether {switch2_iface1_MAC}')

In [27]:
switch2 = slice.get_node(name = 'switch2')

switch2_switch3_subnet = "192.168.5.0/24"
switch2_ip2 = '192.168.5.1/24'
switch3_ip1 = '192.168.5.10/24'

stdout, stderr = switch2.execute(f'sudo ifconfig {switch2_iface2_name} {switch2_ip2}')
stdout, stderr = switch3.execute(f'sudo ifconfig {switch3_iface1_name} {switch3_ip1}')

stdout, stderr = switch2.execute(f'sudo ifconfig {switch2_iface2_name} hw ether {switch2_iface2_MAC}')
stdout, stderr = switch3.execute(f'sudo ifconfig {switch3_iface1_name} hw ether {switch3_iface1_MAC}')

In [28]:
command = 'sudo sysctl -w net.ipv4.ip_forward=1' 
stdout, stderr = switch1.execute(command, quiet=True)
stdout, stderr = switch2.execute(command, quiet=True)
stdout, stderr = switch3.execute(command, quiet=True)

In [29]:
stdout, stderr = switch1.execute(f'sudo ip route del {client_switch1_subnet}', quiet=True)

stdout, stderr = switch2.execute(f'sudo ip route del {switch2_switch3_subnet}', quiet=True)

stdout, stderr = switch3.execute(f'sudo ip route del {switch2_switch3_subnet}', quiet=True)
stdout, stderr = switch3.execute(f'sudo ip route del {client_switch3_subnet}', quiet=True)

In [30]:
gw1 = switch1_ip2.split('/')[0]
gw2 = switch2_ip1.split('/')[0]
gw3 = switch2_ip1.split('/')[0]

stdout, stderr = server.execute(f'sudo ip route add {client_switch1_subnet} via {gw1}')
# stdout, stderr = server.execute(f'sudo ip route add {switch2_switch3_subnet} via {gw2}')
# stdout, stderr = server.execute(f'sudo ip route add {client_switch3_subnet} via {gw3}')

In [31]:
stdout, stderr = server.execute(f'sudo arp -s {gw1} {switch1_iface2_MAC}')
# stdout, stderr = server.execute(f'sudo arp -s {gw2} {switch2_iface1_MAC}')

### File uploads from here

In [32]:
switch1.upload_directory('p4/', 'p4/')
stdout, stderr = switch1.execute(f'p4c p4/main-Copy1.p4')
stdout, stderr = switch1.execute(f'sudo pkill simple_switch')
print(f'sudo simple_switch -i 0@{switch1_iface1_name} -i 1@{switch1_iface2_name} -i 2@{switch1_iface3_name} main-Copy1.json --log-console')
stdout, stderr = switch1.execute(f'sudo simple_switch -i 0@{switch1_iface1_name} -i 1@{switch1_iface2_name} -i 2@{switch1_iface3_name} main-Copy1.json &')

switch2.upload_directory('p4/', 'p4/')
stdout, stderr = switch2.execute(f'p4c p4/main.p4')
stdout, stderr = switch2.execute(f'sudo pkill simple_switch')
print(f'sudo simple_switch -i 0@{switch2_iface2_name} -i 1@{switch2_iface1_name} main.json --log-console')
stdout, stderr = switch2.execute(f'sudo simple_switch -i 0@{switch2_iface2_name} -i 1@{switch2_iface1_name} main.json &')

switch3.upload_directory('p4/', 'p4/')
stdout, stderr = switch3.execute(f'p4c p4/main.p4')
stdout, stderr = switch3.execute(f'sudo pkill simple_switch')
print(f'sudo simple_switch -i 0@{switch3_iface1_name} -i 1@{switch3_iface2_name} main.json --log-console')
stdout, stderr = switch3.execute(f'sudo simple_switch -i 0@{switch3_iface2_name} -i 1@{switch3_iface1_name} main.json &')

sudo simple_switch -i 0@enp8s0 -i 1@enp9s0 -i 2@enp7s0 main-Copy1.json --log-console
Calling target program-options parser
Adding interface enp8s0 as port 0
Adding interface enp9s0 as port 1
Adding interface enp7s0 as port 2
sudo simple_switch -i 0@enp6s0 -i 1@enp7s0 main.json --log-console
Calling target program-options parser
Adding interface enp6s0 as port 0
Adding interface enp7s0 as port 1
sudo simple_switch -i 0@enp6s0 -i 1@enp7s0 main.json --log-console
Calling target program-options parser
Adding interface enp7s0 as port 0
Adding interface enp6s0 as port 1


In [33]:
server.upload_file('p4/interactions.json', 'interactions.json')
server.upload_file('p4/send.py', 'send.py')
client.upload_file('p4/recv.py', 'recv.py')
client.upload_file('p4/anlyz.py', 'anlyz.py')

<SFTPAttributes: [ size=401 uid=1000 gid=1000 mode=0o100664 atime=1701753601 mtime=1701753601 ]>

In [34]:
server.upload_file('p4/send-Copy1.py', 'send-Copy1.py')
server.upload_file('p4/interactions-Copy1.json', 'interactions-Copy1.json')

<SFTPAttributes: [ size=110 uid=1000 gid=1000 mode=0o100664 atime=1701753604 mtime=1701753604 ]>

In [35]:
print('server ssh: {}'.format(server.get_ssh_command()))
print('client ssh: {}'.format(client.get_ssh_command()))
print('switch1 ssh: {}'.format(switch1.get_ssh_command()))
print('switch2 ssh: {}'.format(switch2.get_ssh_command()))
print('switch3 ssh: {}'.format(switch3.get_ssh_command()))

server ssh: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.84
client ssh: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fe97:ec40
switch1 ssh: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@63.239.135.97
switch2 ssh: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.169
switch3 ssh: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.190


### NOTE THAT RULES.SH SHOULD ONLY BE DONE (below block) AFTER SIMPLE_SWITCH COMMAND IS ISSUED

In [36]:
switch1.upload_file('p4/rules1-Copy1.sh', 'rules1-Copy1.sh')
stdout, stderr = switch1.execute('chmod +x rules1-Copy1.sh && ./rules1-Copy1.sh')

switch2.upload_file('p4/rules2-Copy1.sh', 'rules2-Copy1.sh')
stdout, stderr = switch2.execute('chmod +x rules2-Copy1.sh && ./rules2-Copy1.sh')

switch3.upload_file('p4/rules3-Copy1.sh', 'rules3-Copy1.sh')
stdout, stderr = switch3.execute('chmod +x rules3-Copy1.sh && ./rules3-Copy1.sh')

Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:01:00/24
action:              MyIngress.forward
runtime data:        00:00:00:00:00:01	00:00
Entry has been added with handle 0
RuntimeCmd: 
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:02:00/24
action:              MyIngress.forward
runtime data:        00:00:00:00:00:02	00:01
Entry has been added with handle 1
RuntimeCmd: 
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:05:00/24
action:              MyIngress.forward
runtime data:        00:00:00:00:00:07	00:01
Entry has been added with handle 2
RuntimeCmd: 
Obtaining JSON from switch...
Done
Contr

In [37]:
# Download video data
server.execute('mkdir data', quiet = True)
server.execute('wget https://nyu.box.com/shared/static/d6btpwf5lqmkqh53b52ynhmfthh2qtby.tgz -O media.tgz', quiet = True)
server.execute('sudo tar -v -xzf media.tgz -C ./data', quiet = True)

KeyboardInterrupt: 

In [64]:
target_vid = '/home/ubuntu/data/media/BigBuckBunny/4sec/bunny_217761bps'

client.execute('rm -f ~/tmp_f.mp4', quiet = True)
# cat test.mp4 $(ls -vx test*.m4s) > tmp.mp4
client.execute(f'cat test.mp4 $(ls -vx test*.m4s) > tmp.mp4', quiet = True)
# ffmpeg -i  tmp.mp4 -c copy tmp_f.mp4
client.execute('ffmpeg -i  tmp.mp4 -c copy tmp_f.mp4', quiet = True)

('',
 "ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers\n  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)\n  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libv

### ADD CODE HERE TO SEND FILES TO CLIENT

In [65]:
client.download_file("/home/fabric/work/tmp_f.mp4", "/home/ubuntu/tmp_f.mp4")
from IPython.display import Video
Video("/home/fabric/work/tmp_f.mp4", embed=True)